## Day 3: Spiral Memory

http://adventofcode.com/2017/day/3

### Part 1

We're asked to find the Manhattan distance from a positive integer to 1 in the infinite spiral format below.

`17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...`

Use standard coordinates with 1 at the origin, (0, 0), and derive the coordinates of the number.  The first thing to note is that the bottom right corners of each layer of the spiral are squares of successive odd numbers. 1 is at (0, 0), 9 at (1, -1), 25 at (2, -2) and so on. So find the lowest odd number above the square root of the number in question and we can find the layer. Then work backwards round the spiral from the squared odd number.

In [1]:
import math
import numpy as np

def odd_ceil(x):
    ceil_x = math.ceil(x)
    if ceil_x % 2 == 0:
        ceil_x += 1
    return ceil_x

def spiral_coordinate(x):
    odd_above_sqrt = odd_ceil(math.sqrt(x))
    layer = odd_above_sqrt // 2
    
    # Start at the bottom right
    coord = np.array([layer, -layer])
    n_at_coord = odd_above_sqrt ** 2
    
    # Reverse around the spiral until reaching x
    for direction in [np.array(v) for v in [(-1, 0), (0, 1), (1, 0), (0, -1)]]:
        steps = min(layer * 2, n_at_coord - x)
        n_at_coord -= steps
        coord += direction * steps
        
    return tuple(coord)

def spiral_manhattan_from_1(x):
    c = spiral_coordinate(x)
    return abs(c[0]) + abs(c[1])

In [2]:
assert spiral_manhattan_from_1(1) == 0
assert spiral_manhattan_from_1(12) == 3
assert spiral_manhattan_from_1(23) == 2
assert spiral_manhattan_from_1(1024) == 31

In [3]:
INPUT = 361527

spiral_manhattan_from_1(INPUT)

326

### Part 2

We now need to build the spiral from 1 at the origin by summing adjacent values in all directions, including diagonals. The above approach to Part 1 turns out to be an advantage as the `spiral_coordinate` function can be used to build a dictionary relatively easily.

See if you can spot this year's first hack around unhashable mutable data structures.

In [4]:
from itertools import count

def first_value_larger(x):
    spiral = {(0, 0):1}
    
    for n in count(2):
        spiral[spiral_coordinate(n)] = \
            sum(spiral.get(tuple(np.array(spiral_coordinate(n)) + direction), 0) 
                for direction in [np.array(v) for v in [(-1, 0), (0, 1), (1, 0), (0, -1),
                                                        (-1, -1), (-1, 1), (1, -1), (1, 1)]])
        if spiral[spiral_coordinate(n)] > x:
            return spiral[spiral_coordinate(n)]

In [5]:
first_value_larger(INPUT)

363010